## 多线程VS多进程
* 程序的一个运行叫一个进程
* 一个进程里面有多个线程
* 共享互斥问题
* 一个进程多个线程间共享数据和上下文运行环境
* 进程：程序的一次执行（程序装载入内存，系统分配资
    源运行）。
* 每个进程有自己的内存空间、数据栈等，只能使用进
  程间通讯，而不能直接共享信息。
* 线程：所有线程运行在同一个进程中，共享相同的运行
  环境。
* 每个独立的线程有一个程序运行的入口、顺序执行
  序列和程序的出口。
* 线程的运行可以被抢占（中断），或暂时被挂起
  （睡眠），让其他线程运行（让步）。
*  一个进程中的各个线程间共享同一片数据空间。

In [ ]:
import time
import _thread as thread

def loop1():
    print("start loop at :", time.ctime())
    time.sleep(4)
    print("end loop at: ", time.ctime())
    
def loop2():
    # ctime 获取当前时间
    print("start loop at :", time.ctime())
    # 睡眠多长时间，单位秒
    time.sleep(2)
    print("end loop at: ", time.ctime())
    
def main():
    print("start at:", time.ctime)
    #启动多线程的意思是用多线程去执行某个函数
    # 启动多线程函数为start_new_thread
    #参数两个，一个是需要运行的函数，第二个是函数的参数作为元组使用
    # 为空则使用空元祖
    # 注意如果函数只有一个参数，需要参数后有一个逗号
    thread.start_new_thread(loop1,())
    thread.start_new_thread(loop2,())
    print("All done at :", time.ctime())
    
if __name__ == "__main__":
    main()
    while True:
        time.sleep(1)

In [ ]:
import time
import _thread as thread

def loop1(in1):
    print("start loop at :", time.ctime())
    print("我是参数",in1)
    time.sleep(4)
    print("end loop at: ", time.ctime())
    
def loop2(in1,in2):
    # ctime 获取当前时间
    print("start loop at :", time.ctime())
    # 睡眠多长时间，单位秒
    print("我是参数 ",in1,"和参数 ",in2)
    time.sleep(2)
    print("end loop at: ", time.ctime())
    
def main():
    print("start at:", time.ctime)
    #启动多线程的意思是用多线程去执行某个函数
    # 启动多线程函数为start_new_thread
    #参数两个，一个是需要运行的函数，第二个是函数的参数作为元组使用
    # 为空则使用空元祖
    # 注意如果函数只有一个参数，需要参数后有一个逗号
    thread.start_new_thread(loop1,("王老大",))
    thread.start_new_thread(loop2,("王大鹏","王晓鹏",))
    print("All done at :", time.ctime())
    
if __name__ == "__main__":
    main()
    while True:
        time.sleep(10)

start at: <built-in function ctime>
All done at :start loop at :start loop at : Thu Jul 19 21:37:53 2018 Thu Jul 19 21:37:53 2018
我是参数  王大鹏 和参数 
我是参数 王老大
 王晓鹏
 Thu Jul 19 21:37:53 2018
end loop at:  Thu Jul 19 21:37:55 2018
end loop at:  Thu Jul 19 21:37:57 2018


##  threading 的使用
 *  t = threading.Thread(target=xxx, arg=(xxx,))
 *  t.start() 启动多线程
 *  t.join()  等待多线程执行完


In [ ]:
# 比较实用的代码 作业
# 企业里面常用

import threading
from time import sleep, ctime


loop = [4,2]

class ThreadFunc:

    def __init__(self, name):
        self.name = name

    def loop(self, nloop, nsec):
        '''
        :param nloop: loop函数的名称
        :param nsec: 系统休眠时间
        :return:
        '''
        print('Start loop ', nloop, 'at ', ctime())
        sleep(nsec)
        print('Done loop ', nloop, ' at ', ctime())

def main():
    print("Starting at: ", ctime())

    # ThreadFunc("loop").loop 跟一下两个式子相等：
    # t = ThreadFunc("loop")
    # t.loop
    # 以下t1 和  t2的定义方式相等
    t = ThreadFunc("loop")
    t1 = threading.Thread( target = t.loop, args=("LOOP1", 4))
    # 下面这种写法更西方人，工业化一点
    t2 = threading.Thread( target = ThreadFunc('loop').loop, args=("LOOP2", 2))

    # 常见错误写法
    #t1 = threading.Thread(target=ThreadFunc('loop').loop(100,4))
    #t2 = threading.Thread(target=ThreadFunc('loop').loop(100,2))

    t1.start()
    t2.start()

    t1.join( )
    t2.join()


    print("ALL done at: ", ctime())


if __name__ == '__main__':
    main()

In [5]:
import threading

sum = 0
loopSum = 1000000


lock = threading.Lock()


def myAdd():
    global  sum, loopSum

    for i in range(1, loopSum):
        # 上锁，申请锁
        lock.acquire()
        sum += 1
        # 释放锁
        lock.release()


def myMinu():
    global  sum, loopSum
    for i in range(1, loopSum):
        lock.acquire()
        sum -= 1
        lock.release()

if __name__ == '__main__':
    print("Starting ....{0}".format(sum))

    # 开始多线程的实例，看执行结果是否一样
    t1 = threading.Thread(target=myAdd, args=())
    t2 = threading.Thread(target=myMinu, args=())

    t1.start()
    t2.start()

    t1.join()
    t2.join()

    print("Done .... {0}".format(sum))


Starting ....0
Done .... 0


In [6]:
#encoding=utf-8
import threading
import time

# Python2
# from Queue import Queue

# Python3
import queue


class Producer(threading.Thread):
    def run(self):
        global queue
        count = 0
        while True:
            # qsize返回queue内容长度
            if queue.qsize() < 1000:
                for i in range(100):
                    count = count +1
                    msg = '生成产品'+str(count)
                    # put是网queue中放入一个值
                    queue.put(msg)
                    print(msg)
            time.sleep(0.5)


class Consumer(threading.Thread):
    def run(self):
        global queue
        while True:
            if queue.qsize() > 100:
                for i in range(3):
                    # get是从queue中取出一个值
                    msg = self.name + '消费了 '+queue.get()
                    print(msg)
            time.sleep(1)


if __name__ == '__main__':
    queue = queue.Queue()

    for i in range(500):
        queue.put('初始产品'+str(i))
    for i in range(2):
        p = Producer()
        p.start()
    for i in range(5):
        c = Consumer()
        c.start()

生成产品1
生成产品2
生成产品3
生成产品4
生成产品5
生成产品6
生成产品7
生成产品8
生成产品9
生成产品10
生成产品11
生成产品12
生成产品13
生成产品14
生成产品15
生成产品16
生成产品17
生成产品18
生成产品19
生成产品20
生成产品21
生成产品22
生成产品23
生成产品24
生成产品25
生成产品26
生成产品27
生成产品28生成产品1
生成产品29
生成产品30
生成产品31
生成产品32
生成产品33
生成产品34
生成产品35
生成产品36
生成产品37
生成产品38
生成产品39
生成产品40
生成产品41
生成产品42
生成产品43
生成产品44
生成产品45
生成产品46
生成产品47
生成产品48
生成产品49
生成产品50
生成产品51
生成产品52
生成产品53
生成产品54
生成产品55
生成产品56
生成产品57
生成产品58
生成产品59
生成产品60
生成产品61
生成产品62
Thread-10消费了 初始产品0
生成产品63
生成产品64
生成产品65
生成产品66
生成产品67
生成产品68
生成产品69
生成产品70
生成产品71
生成产品72
生成产品73
生成产品74
生成产品75
生成产品76
生成产品77
生成产品78
生成产品2生成产品79Thread-11消费了 初始产品1


Thread-11消费了 初始产品2
Thread-11消费了 初始产品3
生成产品80Thread-12消费了 初始产品4

生成产品3Thread-10消费了 初始产品5
Thread-10消费了 初始产品6

生成产品4
生成产品5
生成产品6
生成产品7
生成产品8
生成产品9
生成产品10
生成产品11
生成产品12
生成产品13
生成产品14
生成产品15
生成产品16
生成产品17
生成产品18
生成产品19
生成产品20
生成产品21
生成产品22
生成产品23
生成产品24
生成产品25
Thread-12消费了 初始产品7生成产品81
Thread-12消费了 初始产品8

生成产品26

生成产品82
生成产品83
生成产品84
生成产品85
生成产品86
生成产品87
生成产品88
生成产品89
生成产品90
生成产品91
生成产品92生成产品27
生成产品28
生成产品29
生

Thread-11消费了 初始产品150
Thread-11消费了 初始产品151
Thread-11消费了 初始产品152
Thread-12消费了 初始产品153
Thread-12消费了 初始产品154
Thread-10消费了 初始产品155Thread-12消费了 初始产品156

Thread-10消费了 初始产品157
Thread-10消费了 初始产品158
Thread-13消费了 初始产品159
Thread-13消费了 初始产品160
Thread-13消费了 初始产品161
Thread-14消费了 初始产品162
Thread-14消费了 初始产品163
Thread-14消费了 初始产品164
Thread-11消费了 初始产品165
Thread-11消费了 初始产品166
Thread-11消费了 初始产品167
Thread-12消费了 初始产品168
Thread-12消费了 初始产品169
Thread-12消费了 初始产品170
Thread-10消费了 初始产品171
Thread-10消费了 初始产品172
Thread-10消费了 初始产品173
Thread-13消费了 初始产品174
Thread-13消费了 初始产品175
Thread-13消费了 初始产品176
Thread-14消费了 初始产品177
Thread-14消费了 初始产品178
Thread-14消费了 初始产品179
Thread-11消费了 初始产品180
Thread-11消费了 初始产品181
Thread-11消费了 初始产品182
Thread-10消费了 初始产品183Thread-12消费了 初始产品184

Thread-10消费了 初始产品185
Thread-10消费了 初始产品186
Thread-12消费了 初始产品187
Thread-12消费了 初始产品188
Thread-13消费了 初始产品189
Thread-13消费了 初始产品190
Thread-13消费了 初始产品191
Thread-14消费了 初始产品192
Thread-14消费了 初始产品193
Thread-14消费了 初始产品194
Thread-11消费了 初始产品195
Thread-11消费了 初始产品196
Thread-11消费了 

Thread-11消费了 初始产品435
Thread-11消费了 初始产品436
Thread-11消费了 初始产品437
Thread-12消费了 初始产品438
Thread-12消费了 初始产品439
Thread-12消费了 初始产品440
Thread-10消费了 初始产品441
Thread-10消费了 初始产品442
Thread-10消费了 初始产品443
Thread-13消费了 初始产品444
Thread-13消费了 初始产品445
Thread-13消费了 初始产品446
Thread-14消费了 初始产品447
Thread-14消费了 初始产品448
Thread-14消费了 初始产品449
Thread-11消费了 初始产品450
Thread-11消费了 初始产品451
Thread-11消费了 初始产品452
Thread-12消费了 初始产品453
Thread-12消费了 初始产品454
Thread-12消费了 初始产品455
Thread-10消费了 初始产品456
Thread-10消费了 初始产品457
Thread-10消费了 初始产品458
Thread-13消费了 初始产品459
Thread-13消费了 初始产品460
Thread-13消费了 初始产品461
Thread-14消费了 初始产品462
Thread-14消费了 初始产品463
Thread-14消费了 初始产品464
Thread-11消费了 初始产品465
Thread-11消费了 初始产品466
Thread-11消费了 初始产品467
Thread-12消费了 初始产品468
Thread-12消费了 初始产品469
Thread-12消费了 初始产品470
Thread-10消费了 初始产品471
Thread-10消费了 初始产品472
Thread-10消费了 初始产品473
Thread-13消费了 初始产品474
Thread-13消费了 初始产品475
Thread-13消费了 初始产品476
Thread-14消费了 初始产品477
Thread-14消费了 初始产品478
Thread-14消费了 初始产品479
Thread-11消费了 初始产品480
Thread-11消费了 初始产品481
Thread-11消费了 

In [1]:
import threading
import time

lock_1 = threading.Lock()
lock_2 = threading.Lock()

def func_1():
    print("func_1 starting.........")
    lock_1.acquire(timeout=4)
    print("func_1 申请了 lock_1....")
    time.sleep(2)
    print("func_1 等待 lock_2.......")

    rst = lock_2.acquire(timeout=2)
    if rst:
        print("func_1 已经得到锁 lock_2")
        lock_2.release()
        print("func_1 释放了锁 lock_2")
    else:
        print("func_1 注定没申请到lock_2.....")

    lock_1.release()
    print("func_1 释放了 lock_1")

    print("func_1 done..........")


def func_2():
    print("func_2 starting.........")
    lock_2.acquire()
    print("func_2 申请了 lock_2....")
    time.sleep(4)
    print("func_2 等待 lock_1.......")
    lock_1.acquire()
    print("func_2 申请了 lock_1.......")

    lock_1.release()
    print("func_2 释放了 lock_1")

    lock_2.release()
    print("func_2 释放了 lock_2")

    print("func_2 done..........")

if __name__ == "__main__":

    print("主程序启动..............")
    t1 = threading.Thread(target=func_1, args=())
    t2 = threading.Thread(target=func_2, args=())

    t1.start()
    t2.start()

    t1.join()
    t2.join()

    print("主程序结束..............")


主程序启动..............
func_1 starting.........
func_1 申请了 lock_1....
func_2 starting.........
func_2 申请了 lock_2....
func_1 等待 lock_2.......
func_1 注定没申请到lock_2.....
func_1 释放了 lock_1
func_1 done..........
func_2 等待 lock_1.......
func_2 申请了 lock_1.......
func_2 释放了 lock_1
func_2 释放了 lock_2
func_2 done..........
主程序结束..............


### 线程替代方案
 - subprocess
      * 完全跳过线程，使用进程
      * 是派生进程的主要替代方案
      * python2.4后映入
      * multiprocessiong
      - 使用threadiing借口派生，使用子进程
      - 允许为多核或者多cpu派生进程，接口跟threading非常相似
      - python2.6
    
- concurrent.futures
    - 新的异步执行模块
    - 任务级别的操作
    - python3.2后引入
### 多进程
- 进程间通讯(InterprocessCommunication, IPC )
- 进程之间无任何共享状态
- 进程的创建
    - 直接生成Process实例对象， 案例19
    - 派生子类， 案例20
    
- 在os中查看pid，ppid以及他们的关系              
    - 案例21
- 生产者消费者模型
    - JoinableQueue
    - 案例22
    - 队列中哨兵的使用, 案例23 
    - 哨兵的改进， 案例24

In [5]:
from multiprocessing import Process
import time
def f(n):
    time.sleep(1)
    print(n*n)
if __name__ =="__main__":
    for i in range(10):
        p = Process(target = f,args = [i,])
        p.start()

In [4]:
import threading
import time
def fun1():
    print("函数一启动")
    time.sleep(3)
    print("结束1")
def fun2():
    print("函数二启动")
    time.sleep(2)
    print("等待函数一")
    print("结束2")
def main():
    print("主程序启动")
    t1 = threading.Thread(target=fun1,args=())
    t2 = threading.Thread(target=fun2,args=())
    
    t1.start()
    t2.start()
    
    t1.join()
    t2.join()
    
    print("主程序结束")

    
if __name__ == "__main__":
    main()

主程序启动
函数一启动
函数二启动
等待函数一
结束2
结束1
主程序结束
